In [1]:
import numpy as np
import cv2
import glob

# we will find a 7 by 7 chess grid
gridDim = 7

# setting termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# preparing object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((gridDim * gridDim, 3), np.float32)
objp[:,:2] = np.mgrid[0 : gridDim, 0 : gridDim].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('calibrationPhotos/*.jpg')

# calculating intrinsic parameters in pixels
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Finding the 7x7 chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (gridDim, gridDim), None)

    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (gridDim, gridDim), corners2, ret)
        img = cv2.resize(img, (540, 960))
        '''
        cv2.imshow(fname, img)
        cv2.waitKey(00)
        cv2.destroyAllWindows()
        '''


In [2]:
# getting camera matrics, rotation and translation vectors, and also distortion coefficients
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print(ret)
print(mtx)
print(dist)
print(rvecs)
print(tvecs)


1.5350202492570277
[[2.96963278e+03 0.00000000e+00 1.03109536e+03]
 [0.00000000e+00 2.99321282e+03 1.72815405e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[ 0.13964465 -2.94796829 -0.03509533 -0.0134911  13.11667691]]
[array([[-0.46157642],
       [-0.07047107],
       [ 0.56028886]]), array([[-0.33540015],
       [ 0.36761341],
       [ 1.43092637]]), array([[ 0.28458689],
       [-0.82049503],
       [ 1.49213999]]), array([[-1.11850531],
       [-0.41396707],
       [-0.04135593]]), array([[-0.15340126],
       [-0.18482341],
       [ 0.21639658]]), array([[ 0.51920667],
       [-0.75424544],
       [ 1.3707366 ]]), array([[-0.77186802],
       [ 0.0816996 ],
       [ 0.13779301]]), array([[ 0.71183341],
       [-0.46406707],
       [ 0.45983717]]), array([[-0.87693091],
       [-0.04632117],
       [ 0.09486029]]), array([[ 0.14058788],
       [-0.66535769],
       [ 0.84298798]]), array([[-0.53810913],
       [-0.0302962 ],
       [ 0.15531315]]), array([[-0.01495586],
 

In [3]:
# building a 3x3 rotation matrices from rotation vectors
rmatrices = np.zeros((111, 3, 3), np.float32)
for i in range(len(rvecs)):
    rmatrices[i], _ = cv2.Rodrigues(rvecs[i])
print(rmatrices)

[[[ 8.47498953e-01 -4.96364236e-01 -1.88064098e-01]
  [ 5.27475417e-01  7.47986674e-01  4.02846813e-01]
  [-5.92893139e-02 -4.40611422e-01  8.95737886e-01]]

 [[ 1.02059938e-01 -9.93767440e-01  4.48332019e-02]
  [ 8.92320275e-01  1.11376457e-01  4.37446892e-01]
  [-4.39713836e-01 -4.64022579e-03  8.98125947e-01]]

 [[-1.23657107e-01 -9.44307685e-01 -3.04945707e-01]
  [ 7.63339400e-01  1.05834328e-01 -6.37269199e-01]
  [ 6.34051979e-01 -3.11579943e-01  7.07740128e-01]]

 [[ 9.23255444e-01  2.37524077e-01 -3.01963031e-01]
  [ 1.73092872e-01  4.44514871e-01  8.78888726e-01]
  [ 3.42984289e-01 -8.63706470e-01  3.69287074e-01]]

 [[ 9.59857881e-01 -1.98593602e-01 -1.98074296e-01]
  [ 2.26699665e-01  9.65125620e-01  1.30918965e-01]
  [ 1.65166900e-01 -1.70566976e-01  9.71404552e-01]]

 [[ 2.93262322e-02 -9.84318018e-01 -1.73948154e-01]
  [ 6.73733771e-01  1.48017451e-01 -7.23998368e-01]
  [ 7.38391995e-01 -9.59626064e-02  6.67509139e-01]]

 [[ 9.87819970e-01 -1.53889462e-01  2.30152886e-02]


In [4]:
# cup photo
img = cv2.imread('cupPhoto/IMG_20201003_163651.jpg', 0)
height, width = img.shape
print(height, width)

# found manually with croping from original image
cellImg = img[1984 : height - 1908, 522 : width - 1620]
cellHeightInPixels, cellWidthInPixels = cellImg.shape
cupImg = img[1739: height - 1716, 1456 : width - 249]
cupHeightInPixels, cupWidthInPixels = cupImg.shape
'''
counter = 1
for i in range(10):
    cupImg = img[1739: height - 1716, 1456 + counter : width - 249]
    cv2.imshow(str(counter), cupImg)
    counter += 1
    cv2.waitKey(5000)
    cv2.destroyAllWindows()
'''

# real size of calibration cessboard cell in mm, parameter can be get during calibration
cellRealHeight = cellRealwidth = 22

# coefficient of relationship a real size to size in pixels
realToPixelCoef = cellRealHeight / cellHeightInPixels

# calculating real size of an object
cupRealHeight = cupHeightInPixels * realToPixelCoef
cupRealWidth = cupWidthInPixels * realToPixelCoef
print(cupRealHeight, cupRealWidth)
print(cupHeightInPixels, cupWidthInPixels)
# 3mm of error in size

# the value is a fy from intrinsic parameters matrix
realDistanseToCup = realToPixelCoef * mtx[1][1]
# 30 mm error
print(realDistanseToCup)



4000 2250
111.01851851851852 111.01851851851852
545 545
609.7285368022168
